<a href="https://colab.research.google.com/github/kthuang20/CancerGenomics/blob/main/Cancer_Genomics_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary files used in this notebook
! gdown --id 1oNNQMVu9FZ-Lg_q26S-cjLGK80Y-emoD # mRNA seq
! gdown --id 1FpKBwcRz1ujdil4IaiNlmAjFdb0PZNjd # methylation
! gdown --id 1LOCgSeKXS-GPgZIff3qA4VhhFKMPDOpt # CNA
! gdown --id 1B1b_2qiFPtnLviL4npRLHNLpkWz1jFAI # gene list
! gdown --id 1jZyU7nMRPMVLJmg02nZ14BQA6_mVWgod # mutations
! gdown --id 1EU9aIaCcm-qB1jEe8hK25bgRCdhIefiS # survival outcomes
! gdown --id 1NEJG2KPBkEssCJ-kbtcbWchweJ7ZPkD0 # log2(CNA)

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1oNNQMVu9FZ-Lg_q26S-cjLGK80Y-emoD
To: /content/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt
100% 26.8M/26.8M [00:00<00:00, 65.9MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1FpKBwcRz1ujdil4IaiNlmAjFdb0PZNjd
To: /content/data_methylation_hm27_hm450_merged.txt
100% 77.3M/77.3M [00:00<00:00, 95.8MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.


In [2]:
# install dash-bio
!pip install dash-bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.5/278.5 kB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.0 MB/s eta 0:00:00
  Created wheel for dash-bio: filename=dash_bio-1.0.2-py3-none-any.whl size=10338086 sha256=3bb731db216f148a175e6b6e0f3634ea770f4b632e51ca07a545063ed0dbcde8
  Stored in directory: /root/.cache/pip/wheels/cb/97/9c/49463914611cb715a46d21a3896d9a886318ea0c451e840196
  Created wheel for GEOparse: filename=GEOparse-2.0.3-py3-none-any.whl size=29046 sha256

In [3]:
# import necessary packages
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import plotly.express as px
import dash_bio

### 1. Pancreatic Adenocarcinoma and Wnt Pathway
*a. Using the CDC website, research the number of people who develop this cancer in the USA and other relevant information*

> In 2020, there were 52,747 new cases of pancreatic cancer and 46,774 deaths from it. In other words, for every 100,000 people, 13 new cases of pancreatic cancer were found and 11 people died from it [[1]](https://gis.cdc.gov/Cancer/USCS/#/AtAGlance/). Pancreatic adenocarcinoma is the most common type of cancer, accounting for 95% of exocrine pancreatic cancer, and therefore will be the focus of this project [[2]](https://www.cancer.org/cancer/types/pancreatic-cancer/about/what-is-pancreatic-cancer.html).

*b. Describe how your pathway can impact cancer.*

> In this pathway's inactivated state, β-catenin protein is normally broken down and degraded through the ubiquitin-proteasomal pathway, keeping levels of β-catenin levels low. This keeps transcription factors inhibited from binding to the DNA to transcribe genes associated with cell proliferation and growth. However, when this pathway is activated, Wnt acts to prevent β-catenin from being degraded, increasing β-catenin levels. β-catenin can then bind to the transcription factors, promoting the transcription of these genes [[3]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5357762/).
>
> Therefore, mutations that lead to sustained activation of the pathway or prevent the degradation of β-catenin can result in increased levels of β-catenin. This excess β-catenin can then bind to transcription factors and promote the transcription of genes involved in cell proliferation and growth, ultimately leading to uncontrolled cell growth and cancer.

### 2. Pancreatic Adencarcinoma PanCancer Atlas Dataset
What are the **observations** and what are the **variables** for each observation? How many are there of each?
> This project uses TCGA data on Pancreatic Adenocarcinoma generated by the PanCancer Atlas [[4]](https://www.cbioportal.org/study/summary?id=paad_tcga_pan_can_atlas_2018). The dataset contains a total of 184 tumor samples (observations). The following was measured from the samples:
> * mRNA expression levels of each of 19,893 genes (measured for 177/184 samples)
> * methlyation levels of 22,601 genomic regions (measured for all 184 samples)
> * the mutations of each of the 184 samples in each gene (if any)
> * copy number alterations in each gene (done for 183/184 samples)


In [4]:
# location of file containing the mRNA sequencing data for all samples
file_path = 'data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt'
# import the dataset as a dataframe
mRNA_exp = pd.read_table(file_path)
# remove rows without any genes
mRNA_exp = mRNA_exp[mRNA_exp['Hugo_Symbol'].notna()]
# remove rows where there was no expression levels recorded
mRNA_exp = mRNA_exp.loc[mRNA_exp.drop(columns=['Hugo_Symbol','Entrez_Gene_Id']).notna().sum(axis=1) > 0]
mRNA_exp

,Hugo_Symbol,Entrez_Gene_Id,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,TCGA-2J-AABE-01,TCGA-2J-AABF-01,...,TCGA-XD-AAUG-01,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01
2,UBE2Q2P2,100134869,0.0096,0.0599,-0.9737,0.8455,0.8184,1.7167,1.3022,0.6549,...,1.0778,-0.6572,0.2441,0.6851,-0.2044,0.3293,0.0401,-0.8810,1.5454,0.4204
3,HMGB1P1,10357,0.3928,-0.5735,1.8915,-0.0921,-0.4910,-1.1825,0.1399,0.6376,...,-1.4345,-0.6339,-0.3799,0.9538,1.1697,-0.5930,0.2875,0.1249,0.6197,1.2980
7,RNU12-2P,26823,-0.8900,-0.1096,-2.3425,-2.3425,-2.3425,-0.5646,-2.3425,-2.3425,...,0.7834,0.2660,0.1744,-2.3425,1.8053,-0.6336,-2.3425,-2.3425,-0.0309,2.5850
10,EZHIP,340602,-0.7417,-1.4532,-1.4532,-1.4532,0.5981,-1.4532,-1.4532,-0.1561,...,1.6732,0.5628,-0.2203,-1.4532,-0.1633,0.7300,0.5472,-0.6293,-0.3208,2.9710
11,EFCAB8,388795,-0.8963,-0.9661,0.7841,-1.4985,1.3052,-0.2677,-1.4985,0.5388,...,0.1802,-0.4171,1.2052,-0.5812,-0.8565,0.3490,-0.4271,-1.4985,-0.2211,2.5727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A,440590,-1.4824,-1.4824,-0.7256,1.0819,-0.2853,-0.1478,-0.6930,-0.2921,...,-1.4824,-1.4824,0.9774,-1.4824,-0.7863,0.2026,-1.4824,-0.7264,-1.4824,2.2256
20527,ZYG11B,79699,-0.4936,-0.1143,0.0619,0.0479,-0.1151,1.0851,-0.5433,0.2243,...,0.5972,0.6609,-0.3349,0.1784,0.2845,0.0807,0.3530,-0.3165,-2.3282,0.1352
20528,ZYX,7791,0.8177,0.1063,1.7172,0.2075,0.9191,-0.2003,0.7436,0.3015,...,0.8103,0.0656,0.3420,1.7124,0.2205,0.5369,1.1194,1.2689,-0.0816,-0.1359
20529,ZZEF1,23140,-0.1629,-0.6069,-0.7065,-0.8278,-0.1500,0.4317,-0.8149,0.7066,...,-0.0721,0.6847,-0.3864,-0.5566,0.2326,0.3626,-0.4978,-0.6741,-0.9126,1.2289


In [5]:
# location of file containing the methylation of a genomic region of an exome
file_path = 'data_methylation_hm27_hm450_merged.txt'
# import the dataset as a dataframe
methylation_levels = pd.read_table(file_path)
methylation_levels

,ENTITY_STABLE_ID,NAME,DESCRIPTION,TRANSCRIPT_ID,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,...,TCGA-XD-AAUG-01,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01
0,cg00000292,ATP2A1,1stExon,NM_173201;NM_004320,0.733917,0.719068,0.560780,0.670286,0.688073,0.705372,...,0.632310,0.761573,0.644253,0.697230,0.550313,0.736484,0.678476,0.443366,0.634037,0.841216
1,cg00003994,MEOX2,1stExon,NM_005924,0.110034,0.112912,0.103200,0.080924,0.134720,0.060966,...,0.057942,0.071444,0.093027,0.062793,0.066140,0.091232,0.056195,0.218264,0.215530,0.077785
2,cg00005847,HOXD3,5'UTR,NM_006898,0.598123,0.144939,0.471551,0.334088,0.271976,0.497987,...,0.254899,0.373482,0.343299,0.321652,0.356024,0.306038,0.400456,0.549984,0.744128,0.292187
3,cg00007981,PANX1,1stExon,NM_015368,0.025716,0.025669,0.026010,0.025039,0.031081,0.027746,...,0.023778,0.020776,0.024543,0.021864,0.026096,0.031424,0.029742,0.020904,0.031055,0.026724
4,cg00008493,KIAA1409;COX8C,Body;5'UTR,NM_020818;NM_182971,0.947818,0.918361,0.937682,0.934786,0.938346,0.943597,...,0.949865,0.944085,0.942790,0.919763,0.944443,0.933676,0.920512,0.934375,0.958107,0.940709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22596,cg27657283,LEPRE1;C1orf50,Body;TSS1500,NM_022356;NM_001146289;NM_024097,0.032062,0.033511,0.030302,0.031422,0.044181,0.037770,...,0.032174,0.029996,0.029864,0.032304,0.033330,0.042130,0.041855,0.038142,0.038249,0.027484
22597,cg27661264,GNAS,TSS1500;3'UTR,NM_016592;NM_001077490;NM_080425,0.439556,0.224960,0.335165,0.381820,0.312923,0.399568,...,0.333729,0.309576,0.363753,0.319711,0.403687,0.420992,0.382321,0.280691,0.448022,0.329237
22598,cg27662379,RPN1,TSS200,NM_002950,0.087215,0.045312,0.084045,0.075204,0.117227,0.057109,...,0.066277,0.057452,0.099178,0.039511,0.037407,0.043382,0.054221,0.041047,0.043322,0.041996
22599,cg27662877,CYB5A,TSS1500,NM_001914;NM_148923,0.033459,0.037676,0.056683,0.035521,0.042462,0.039009,...,0.040657,0.027856,0.031584,0.027708,0.051507,0.038634,0.044226,0.045798,0.048060,0.028841


In [6]:
# location of file containing the copy number alterations for each gene and sample
file_path = 'data_cna.txt'
# import the dataset as a dataframe
cna_data = pd.read_table(file_path)
cna_data

,Hugo_Symbol,Entrez_Gene_Id,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,TCGA-2J-AABE-01,TCGA-2J-AABF-01,...,TCGA-XD-AAUG-01,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01
0,ACAP3,116983.0,0,0,1,0,0,-1,0,1,...,0,0,0,0,0,0,0,0,-1,0
1,ACTRT2,140625.0,0,0,1,0,0,-1,0,1,...,0,0,0,0,0,0,0,0,-1,0
2,AGRN,375790.0,0,0,1,0,0,-1,0,1,...,0,0,0,0,0,0,0,0,-1,0
3,ANKRD65,441869.0,0,0,1,0,0,-1,0,1,...,0,0,0,0,0,0,0,0,-1,0
4,ATAD3A,55210.0,0,0,1,0,0,-1,0,1,...,0,0,0,0,0,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25123,hsa-mir-513c,NaN,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25124,hsa-mir-513b,NaN,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25125,hsa-mir-513a-1,NaN,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25126,hsa-mir-513a-2,NaN,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. Analyze the Mutations of Genes in Wnt Pathway
***NOTE:*** A list of genes associated with the WNT signaling pathway was downloaded from the Harmonizome API [[5]](https://maayanlab.cloud/Harmonizome/gene_set/Wnt+signaling+pathway/PANTHER+Pathways). I then performed the following command in bash to extract only the names of the genes.
```bash
grep -o '"symbol":"[A-Z0-9]*"' wnt_genes.txt | cut -d '"' -f 4 > wnt_gene_list.txt
```

#### a. Make a figure (such as from cBioPortal) summarizing the mutations

In [7]:
# initialize an empty list to store the words
wnt_genes = []

# read in the text file
with open('wnt_gene_list.txt', 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # add each gene to the running list of genes
        wnt_genes.append(line.strip())

In [8]:
# location of file
file_path = 'data_mutations.txt'
# read in the file as a dataframe
mutations = pd.read_table(file_path)
# create a dataframe containing only the genes that are in the wnt pathway
wnt_mutations = mutations.loc[mutations['Hugo_Symbol'].isin(wnt_genes)]
wnt_mutations.head()

<ipython-input-8-c6f32b76089a>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations = pd.read_table(file_path)


,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,SYMBOL_SOURCE,TREMBL,TSL,UNIPARC,VARIANT_CLASS,all_effects,cDNA_position,n_depth,t_depth,Annotation_Status
19,PCDH9,5101.0,.,GRCh37,13,66878827,66878827,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI00001FCE90,SNV,"PCDH9,missense_variant,p.Ala1191Val,ENST000003...",4366,61,300,SUCCESS
66,PCDHA9,9752.0,.,GRCh37,5,140229874,140229874,+,synonymous_variant,Silent,...,HGNC,.,.,UPI00001273D1,SNV,"PCDHA9,synonymous_variant,p.%3D,ENST0000037812...",2827,75,417,SUCCESS
67,PCDHA9,9752.0,.,GRCh37,5,140230003,140230003,+,synonymous_variant,Silent,...,HGNC,.,.,UPI00001273D1,SNV,"PCDHA9,synonymous_variant,p.%3D,ENST0000037812...",2956,97,404,SUCCESS
91,SMAD4,4089.0,.,GRCh37,18,48584504,48584504,+,frameshift_variant,Frame_Shift_Del,...,HGNC,"Q9BYG6_HUMAN,K7ENG8_HUMAN,K7ENG1_HUMAN,K7EL18_...",.,UPI0000053431,deletion,"SMAD4,frameshift_variant,p.Ser227ValfsTer14,EN...",1215,42,213,SUCCESS
132,PCDH9,5101.0,.,GRCh37,13,66878849,66878849,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI00001FCE90,SNV,"PCDH9,missense_variant,p.Asn1184Tyr,ENST000003...",4344,60,335,SUCCESS


In [9]:
# create a dataframe containing the number of mutations each tumor has
num_mutations = wnt_mutations.groupby('Tumor_Sample_Barcode').size().reset_index()
# rename the columns
num_mutations.columns = ['sample', '# mutations']
# sort samples from greatest to fewest number of mutations
num_mutations.sort_values('# mutations', ascending=False, inplace=True)
num_mutations

,sample,# mutations
91,TCGA-IB-7651-01,502
60,TCGA-HV-A5A3-01,13
65,TCGA-HV-A7OP-01,8
88,TCGA-IB-7644-01,6
29,TCGA-3A-A9J0-01,6
...,...,...
37,TCGA-FB-A5VM-01,1
36,TCGA-FB-A545-01,1
108,TCGA-IB-AAUM-01,1
112,TCGA-IB-AAUR-01,1


In [10]:
# create a histogram showing the frequency of total number of mutations in samples
fig = px.histogram(num_mutations, x='# mutations', nbins=70)

# update labels on figure
fig.update_layout(
    title = 'Figure 1. Frequency of Mutations',
    title_x = 0.5,
    xaxis_title = 'Number of Mutations',
    yaxis_title = 'Number of samples',
    bargap=0.2)

# show figure
fig.show()

#### b. How many patients have each gene mutated with either copy number alterations or point mutations?

In [11]:
# create a dataframe that contains only the copy number alterations in genes in the Wnt pathway
wnt_cnas = cna_data.loc[cna_data['Hugo_Symbol'].isin(wnt_genes)]
# create a dataframe containing whether or not the sample has a copy number alteration at gene
has_cnas = wnt_cnas.drop(columns=['Hugo_Symbol', 'Entrez_Gene_Id'], axis=1).abs()
# add back the column with the names of the genes
has_cnas = pd.concat([wnt_cnas['Hugo_Symbol'], has_cnas], axis=1)
has_cnas

,Hugo_Symbol,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,TCGA-2J-AABE-01,TCGA-2J-AABF-01,TCGA-2J-AABH-01,...,TCGA-XD-AAUG-01,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01
18,DVL1,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
26,GNB1,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
70,PRKCZ,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
353,WNT4,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
442,ARID1A,1,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24354,ARR3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
24389,HDAC8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
24463,PCDH11X,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24470,PCDH19,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
### function to determine if each tumor sample has a point mutation in one gene
def samples_with_mut(muts_in_gene, wnt_mut):
    ## store the name of the gene
    gene = muts_in_gene['Hugo_Symbol']
    ## store the name of all the samples with a point mutation in gene
    samples_with_mut = wnt_mut.loc[wnt_mut['Hugo_Symbol'] == gene, 'Tumor_Sample_Barcode'].unique()
    ## for each sample with a point mutation,
    for sample in samples_with_mut:
        # record that the sample has a mutation in this gene
        muts_in_gene[sample] = 1
    ## fill the remaining samples with 0 (no mutation in gene)
    muts_in_gene.fillna(0, inplace=True)
    return muts_in_gene

In [13]:
# create a dataframe to store whether each tumor sample has a mutation in that gene
has_mut = pd.DataFrame(columns=has_cnas.columns)

# create a dataframe containing only data about point mutations
wnt_point_muts = wnt_mutations[wnt_mutations['Variant_Type'] == 'SNP']
# add the list of genes that have a point mutation
has_mut['Hugo_Symbol'] = wnt_point_muts['Hugo_Symbol'].unique()

# for each gene, add whether each tumor sample has a point mutation at that gene
has_mut = has_mut.apply(lambda row: samples_with_mut(row, wnt_point_muts), axis=1)
has_mut

,Hugo_Symbol,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,TCGA-2J-AABE-01,TCGA-2J-AABF-01,TCGA-2J-AABH-01,...,TCGA-XD-AAUG-01,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01
0,PCDH9,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PCDHA9,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,SRCAP,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SMAD4,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,PCDHA3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,GNB2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
208,HDAC3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209,GNG4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
210,CSNK2B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
### function to determine if each tumor sample has a point mutation or copy number alteration in gene
def mut_or_cna(gene, has_mut, has_cnas):
    ## if there was 1+ samples that had a point mutation and copy number alteration in this gene
    if (gene in has_mut['Hugo_Symbol'].values) and (gene in has_cnas['Hugo_Symbol'].values):
        # store whether each sample has a point mutation in gene
        gene_has_mut = has_mut.loc[has_mut['Hugo_Symbol'] == gene]
        # store whether each sample has a copy number alteration in gene
        gene_has_cna = has_cnas.loc[has_cnas['Hugo_Symbol'] == gene]
        # combine the two into one dataframe
        mut_cna = pd.concat([gene_has_mut, gene_has_cna])
        # store whether each sample had a point mutation or copy number alteration
        mut_or_cna = (mut_cna.drop('Hugo_Symbol', axis=1).sum() > 0).astype(int)
        # return this information
        return mut_or_cna
    ## otherwise, samples only had a copy number alteration in this gene
    elif gene in has_cnas['Hugo_Symbol'].values:
        # store whether each sample has a copy number alteration in gene
        gene_has_cna = has_cnas.loc[has_cnas['Hugo_Symbol'] == gene].drop('Hugo_Symbol', axis=1)
        # return this information
        return gene_has_cna.iloc[0]
    ## otherwise, samples only had point mutations in gene
    else:
        # store whether each sample has a point mutation in gene
        gene_has_mut = has_mut.loc[has_mut['Hugo_Symbol'] == gene].drop('Hugo_Symbol', axis=1)
        # return this information
        return gene_has_mut.iloc[0]

In [15]:
### create a dataframe that contains whether each patient has a copy number alteration or mutation in each gene
has_mut_or_cna = pd.DataFrame(columns=has_mut.columns)
### add a column containing all the genes that have either a point mutation or copy number alteration
genes = list(set(has_mut['Hugo_Symbol']) | set(has_cnas['Hugo_Symbol']))
has_mut_or_cna['Hugo_Symbol'] = genes
### set index as the genes
has_mut_or_cna.set_index('Hugo_Symbol', inplace=True)
### iterate through each gene
for gene in genes:
    ## add whether each sample had a point mutation or copy number alteration in each gene
    gene_data = mut_or_cna(gene, has_mut, has_cnas)
    ## add this information to the dataframe
    has_mut_or_cna.loc[gene] = gene_data

### add a column with the total number of samples with any point mutations or copy alterations in gene
has_mut_or_cna['total # samples'] = has_mut_or_cna.sum(axis=1)
### sort genes from greatest to least amount of mutations in each gene
has_mut_or_cna.sort_values('total # samples', ascending=False, inplace=True)
has_mut_or_cna

,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,TCGA-2J-AABE-01,TCGA-2J-AABF-01,TCGA-2J-AABH-01,TCGA-2J-AABI-01,...,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01,total # samples
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
SMAD4,0,1,1,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,0,136
CDH20,0,1,1,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,0,131
CDH7,0,1,1,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,0,131
CDH19,0,1,1,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,0,131
NFATC1,0,1,1,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,0,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FRZB,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,27
CTNNA2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,27
TCF7L1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,26


In [16]:
# show the total number of samples with a point mutations or copy alteration for each gene
num_samples_with_mut = has_mut_or_cna['total # samples'].reset_index()
# rename columns
num_samples_with_mut.columns = ['gene', 'total # samples']
# add a column with the % of all patients in dataset
num_samples_with_mut['% of all samples'] = num_samples_with_mut['total # samples']/float(184)*100
num_samples_with_mut

,gene,total # samples,% of all samples
0,SMAD4,136,73.913043
1,CDH20,131,71.195652
2,CDH7,131,71.195652
3,CDH19,131,71.195652
4,NFATC1,128,69.565217
...,...,...,...
248,FRZB,27,14.673913
249,CTNNA2,27,14.673913
250,TCF7L1,26,14.130435
251,PPP3R1,25,13.586957


In [17]:
# create a bar chart showing the number of patients with mutations in gene
fig = px.bar(num_samples_with_mut, x='gene', y='total # samples')

# update labels on figure
fig.update_layout(
    title = 'Figure 2. Number of Patients with a Copy Number Alteration or Point Mutation in Gene',
    title_x = 0.5,
    xaxis_title = 'Gene',
    yaxis_title = 'Number of Patients',
    width = 1050
)

# show figure
fig.show()

#### c. For the top 3 most mutated genes in the Wnt pathway:
1. How often is each gene mutated?
2. Are the mutations activating or inactivating?
3. Is the mutation pattern consistent with oncogene or tumor suppressor activity in cancer?

In [18]:
# show the top 4 most mutated genes
num_samples_with_mut.head(4)

,gene,total # samples,% of all samples
0,SMAD4,136,73.913043
1,CDH20,131,71.195652
2,CDH7,131,71.195652
3,CDH19,131,71.195652


In [19]:
### function to analyze types of mutations for one gene
def analyze_mut(gene, wnt_mut, wnt_cna):
    ## create a dataframe containing all of the mutations in gene
    gene_mut = wnt_mutations[wnt_mutations['Hugo_Symbol'] == gene]
    ## select the CNA levels for gene of interest
    gene_cna = wnt_cna[wnt_cna['Hugo_Symbol'] == gene]
    ## have the cna as a column
    gene_cna = gene_cna.iloc[0, 2:].reset_index()
    ## rename columns
    gene_cna.columns = ['sample', 'CNA']
    ## remove numbers at end of patient name
    gene_cna['sample'] = gene_cna['sample'].str[:-3]

    ## create a dataframe showing the number of mutations of each type
    gene_mut_stats = gene_mut['Variant_Classification'].value_counts().reset_index()
    ## rename the column
    gene_mut_stats.columns = ['mutation type', '# samples']
    ## add a column with % of all mutations
    total_mut = gene_mut_stats['# samples'].sum()
    gene_mut_stats['%'] = gene_mut_stats['# samples']/float(total_mut) * 100
    print(f'Summary of Mutations in {gene}')
    print(gene_mut_stats)

    ## create a dataframe with the summary statistics for the copy number alterations
    gene_cna_stats = gene_cna['CNA'].value_counts().reset_index()
    ## rename the columns
    gene_cna_stats.columns = ['CNA type', '# samples']
    ## add a column with % of all copy number alterations
    total_cna = gene_cna_stats['# samples'].sum()
    gene_cna_stats['%'] = gene_cna_stats['# samples']/float(total_cna) * 100
    print(f'\nSummary of Copy Number Alterations in {gene}')
    print(gene_cna_stats)

    return gene_mut, gene_cna

##### *Results*

In [20]:
# get summary of mutations and copy number alterations for the most mutated gene
SMAD4_mut, SMAD4_cna = analyze_mut('SMAD4', wnt_mutations, wnt_cnas)

Summary of Mutations in SMAD4
       mutation type  # samples     %
0  Missense_Mutation         16  40.0
1  Nonsense_Mutation         12  30.0
2    Frame_Shift_Del          7  17.5
3    Frame_Shift_Ins          3   7.5
4        Splice_Site          1   2.5
5       In_Frame_Del          1   2.5

Summary of Copy Number Alterations in SMAD4
   CNA type  # samples          %
0        -1        102  55.737705
1         0         51  27.868852
2        -2         23  12.568306
3         1          7   3.825137


**SMAD4**
> 1. About 74% of all patients had mutations in this gene (copy number alteration or point mutation).
>
> 2. Mutations in this gene seem to be inactivating as there is a high percentage (50%) of mutations that result in a loss of function in the resulting protein. Additionally, it seems that 56% of all patients have a hemizygous deletion for this gene.
>
> 3. Tumor suppressor genes typically act to prevent cancer formation by inhibiting cell growth or promoting apoptosis. This means that inactivating or having deletions in tumor suppressor genes, would promote cell growth. These results are consistent with this gene, suggesting that it is a tumor suppressor.

---

In [21]:
# get summary of mutations and copy number alterations for the 2nd most mutated gene
CDH7_mut, CDH7_cna = analyze_mut('CDH7', wnt_mutations, wnt_cnas)

Summary of Mutations in CDH7
       mutation type  # samples      %
0  Missense_Mutation          1  100.0

Summary of Copy Number Alterations in CDH7
   CNA type  # samples          %
0        -1        117  63.934426
1         0         53  28.961749
2        -2          6   3.278689
3         1          6   3.278689
4         2          1   0.546448


**CDH7**
> 1. About 71% of all patients had mutations in this gene (copy number alteration or point mutation).
>
> 2. The patients predominantly (64% of all patients) have a hemizygous deletion for this gene.
>
> 3. The significant proportion of deletions suggests a loss of gene function, which is more consistent with tumor suppressor activity.

---

In [22]:
# get summary of mutations and copy number alterations for the 3rd most mutated gene
CDH20_mut, CDH20_cna = analyze_mut('CDH20', wnt_mutations, wnt_cnas)

Summary of Mutations in CDH20
       mutation type  # samples          %
0  Missense_Mutation          2  66.666667
1             Silent          1  33.333333

Summary of Copy Number Alterations in CDH20
   CNA type  # samples          %
0        -1        118  64.480874
1         0         53  28.961749
2         1          6   3.278689
3        -2          5   2.732240
4         2          1   0.546448


**CDH20**
> 1. About 71.2% of all patients had mutations in this gene (copy number alteration or point mutation).
>
> 2. The patients predominantly (64% of all patients) have a hemizygous deletion for this gene.
>
> 3. The significant proportion of deletions suggests a loss of gene function, which is more consistent with tumor suppressor activity.

---

#### d. Which of your top 3 mutations is associated with increased or decreased survival outcome?

In [23]:
# location of file containing the survival outcomes of patients
file_path = 'data_clinical_patient.txt'
# import data as a dataframe
patient_info = pd.read_table(file_path, skiprows=4)
# store a dataframe containing only survival outcome
survival_outcome = patient_info[['PATIENT_ID', 'DSS_STATUS']]
# remove samples no status recorded
survival_outcome = survival_outcome[survival_outcome['DSS_STATUS'].notna()]
# convert dss status to numeric values
survival_outcome['DSS_STATUS'] = survival_outcome['DSS_STATUS'].str[0].astype(int)
# store a list of all patients for future reference
all_patients = survival_outcome['PATIENT_ID']
survival_outcome

,PATIENT_ID,DSS_STATUS
0,TCGA-2J-AAB1,1
1,TCGA-2J-AAB4,0
2,TCGA-2J-AAB6,1
3,TCGA-2J-AAB8,0
4,TCGA-2J-AAB9,1
...,...,...
179,TCGA-XN-A8T5,0
180,TCGA-YB-A89D,0
181,TCGA-YH-A8SY,0
182,TCGA-YY-A8LH,0


In [78]:
# store a list containing the top 3 mutated genes in the Wnt pathway
top_genes = ['SMAD4', 'CDH7', 'CDH20']
# store the number of mutations in for each patient for each of the top 3 mutated genes
total_mut_per_patient = has_mut_or_cna.loc[top_genes].drop('total # samples', axis=1).transpose().reset_index()
# rename columns
total_mut_per_patient.columns = ['PATIENT_ID'] + top_genes
# remove the number barcodes to get just the patient id
total_mut_per_patient['PATIENT_ID'] = total_mut_per_patient['PATIENT_ID'].str[:-3]
# add survival outcome information to these patients
total_mut_per_patient = pd.merge(total_mut_per_patient, survival_outcome, on='PATIENT_ID', how='inner')
total_mut_per_patient

,PATIENT_ID,SMAD4,CDH7,CDH20,DSS_STATUS
0,TCGA-2J-AAB1,0,0,0,1
1,TCGA-2J-AAB4,1,1,1,0
2,TCGA-2J-AAB6,1,1,1,1
3,TCGA-2J-AAB8,1,1,1,0
4,TCGA-2J-AAB9,0,0,0,1
...,...,...,...,...,...
171,TCGA-XN-A8T5,0,0,0,0
172,TCGA-YB-A89D,1,1,1,0
173,TCGA-YH-A8SY,1,1,1,0
174,TCGA-YY-A8LH,1,1,1,0


In [25]:
# calculate and show the mutation frequency for each gene separated by survival outcome
mut_freq = total_mut_per_patient.drop('PATIENT_ID', axis=1).groupby('DSS_STATUS').sum()/has_mut_or_cna.shape[1]*100
mut_freq.index = ['Tumor Free', 'With Tumor']
print('Mutation Frequencies for Each Gene Based on Survival Status:')
mut_freq

Mutation Frequencies for Each Gene Based on Survival Status:


,SMAD4,CDH7,CDH20
Tumor Free,38.043478,36.413043,36.413043
With Tumor,32.065217,30.978261,30.978261


Based on the mutation frequencies above, the mutation frequencies are much higher for samples that are tumor free for all of top 3 mutated genes in the Wnt pathway. This suggests that mutations in all three genes might be associated with an increase in survival outcome.

#### e. How often is the pathway mutated overall in this cancer (how many patients have one or more genes mutated)?

In [26]:
# create a dataframe showing number of gene mutated for each patient
num_mutated = wnt_mutations.groupby('Tumor_Sample_Barcode')['Hugo_Symbol'].size().reset_index()
# rename the columns
num_mutated.columns = ['sample', '# genes']
# show the number of samples with 1+ gene mutated
at_least1 = num_mutated[num_mutated['# genes'] >= 1].shape[0]
# show results
print(f'{at_least1}/184 patients have 1 or more genes mutated')

138/184 patients have 1 or more genes mutated


### 4. Gene Expression Analysis
Pick a gene (possibly the same gene) for each of the gene expression analyses below. For each of the two types of (epi)genomic changes listed below, make a figure to visualize the association of the (epi)genomic alteration and gene expression. Then, interpret your figure: this includes stating how the (epi)genomic alteration affects this gene and what we can conclude about which subjects have oncogenic changes to this gene.


#### a. Association of expression and methylation at the locus

In [27]:
# store the name of the gene
gene = 'SMAD4'
# create a dataframe containing the expression levels of a tumor
exp_methyl = pd.DataFrame(columns=['sample',])
# add the sample names
exp_methyl['sample'] = all_patients

# store the mRNA levels for gene
gene_exp = mRNA_exp[mRNA_exp['Hugo_Symbol'] == gene].iloc[0, 2:].reset_index()
# rename columns
gene_exp.columns = ['sample', 'expression']
# remove numbers at the end of sample names
gene_exp['sample'] = gene_exp['sample'].str[:-3]
# add the expression levels for the patients
gene_exp = pd.merge(exp_methyl, gene_exp, on='sample', how='inner')

# store the methylation levels for gene
gene_methyl = methylation_levels[methylation_levels['NAME'] == gene].iloc[0, 2:].reset_index()
# rename columns
gene_methyl.columns = ['sample', 'methylation']
# remove numbers at the end of sample names
gene_methyl['sample'] = gene_methyl['sample'].str[:-3]
# add the expression levels for the patients
exp_methyl = pd.merge(gene_exp, gene_methyl, on='sample', how='inner')
exp_methyl

,sample,expression,methylation
0,TCGA-2J-AAB1,0.6559,0.043253
1,TCGA-2J-AAB4,-0.381,0.050801
2,TCGA-2J-AAB6,-0.921,0.041
3,TCGA-2J-AAB8,-0.5721,0.048676
4,TCGA-2J-AAB9,0.3982,0.054787
...,...,...,...
166,TCGA-XN-A8T5,0.1801,0.052656
167,TCGA-YB-A89D,0.157,0.051189
168,TCGA-YH-A8SY,-0.5816,0.039769
169,TCGA-YY-A8LH,0.1962,0.065629


In [28]:
# create a scatter plot with line of best fit
fig = px.scatter(exp_methyl, x = 'methylation', y = 'expression', trendline = 'ols')

# add labels to figure
fig.update_layout(
    title = 'Figure 3. Association of Expression and Methylation in ' + gene,
    title_x = 0.5,
    xaxis_title = 'Methylation levels of ' + gene,
    yaxis_title = 'mRNA Expression levels of levels of ' + gene,
    height = 500)

# show figure
fig.show()

Based on this scatter plot, the methylation levels of SMAD4 does not seem to affect SMAD4 expression levels.

#### b. Association of expression and copy number at the locus

In [29]:
# add the expression levels for the patients
exp_cna = pd.merge(gene_exp, SMAD4_cna, on='sample', how='inner')
exp_cna

,sample,expression,CNA
0,TCGA-2J-AAB1,0.6559,0
1,TCGA-2J-AAB4,-0.381,-1
2,TCGA-2J-AAB6,-0.921,-1
3,TCGA-2J-AAB8,-0.5721,-2
4,TCGA-2J-AAB9,0.3982,0
...,...,...,...
165,TCGA-XN-A8T5,0.1801,0
166,TCGA-YB-A89D,0.157,-1
167,TCGA-YH-A8SY,-0.5816,-1
168,TCGA-YY-A8LH,0.1962,1


In [30]:
# create a violin plot describing the mRNA expression levels of SMAD4
fig = px.violin(exp_cna, x = 'CNA', y = 'expression', box=True)

# add labels to figure
fig.update_layout(
    title = 'Figure 4. Expression and Copy Number in ' + gene,
    title_x = 0.5,
    xaxis_title = 'Copy Number of ' + gene,
    yaxis_title = 'mRNA Expression levels of ' + gene,
    height = 500)

# show figure
fig.show()

In [31]:
# location to file with the log2 copy number
file_path = 'data_log2_cna.txt'
# read in the file as a dataframe
log2_cna = pd.read_table(file_path)
# log2 CNA levels for gene
gene_log2_cna = log2_cna[log2_cna['Hugo_Symbol'] == gene].iloc[0, 2:].reset_index()
# rename columns
gene_log2_cna.columns = ['sample', 'log2 CNA']
# remove numbers at the end of sample names
gene_log2_cna['sample'] = gene_log2_cna['sample'].str[:-3]
# add the expression levels for the patients
exp_log2_cna = pd.merge(gene_exp, gene_log2_cna, on='sample', how='inner')
exp_log2_cna

,sample,expression,log2 CNA
0,TCGA-2J-AAB1,0.6559,-0.063
1,TCGA-2J-AAB4,-0.381,-0.315
2,TCGA-2J-AAB6,-0.921,-0.587
3,TCGA-2J-AAB8,-0.5721,-0.35
4,TCGA-2J-AAB9,0.3982,-0.094
...,...,...,...
165,TCGA-XN-A8T5,0.1801,-0.076
166,TCGA-YB-A89D,0.157,-0.215
167,TCGA-YH-A8SY,-0.5816,-0.504
168,TCGA-YY-A8LH,0.1962,0.109


In [32]:
# create a scatter plot with line of best fit
fig = px.scatter(exp_log2_cna, x = 'log2 CNA', y = 'expression', trendline = 'ols')

# add labels to figure
fig.update_layout(
    title = 'Figure 5. mRNA Expression and Log-Copy Number in ' + gene,
    title_x = 0.5,
    xaxis_title = 'Log2(Copy Number of ' + gene + ')',
    yaxis_title = 'mRNA Expression levels of levels of ' + gene,
    height = 500)

# show figure
fig.show()

Results show that while DNA methylation levels do not appear to directly influence SMAD4 mRNA expression (Figure 3), increasing copy number (amplification) does increase overall SMHD4 expression levels (Figures 4 and 5). This suggests that patients with an increase in copy number in this gene may be associated with oncogenic changes. Below are the patients that exhibit amplification of this gene.

In [33]:
# show the samples that exhibited amplification of this gene
exp_cna[exp_cna['CNA'] > 0]

,sample,expression,CNA
12,TCGA-2J-AABP,1.184,1
22,TCGA-2L-AAQM,1.306,1
30,TCGA-3A-A9IN,1.7244,1
35,TCGA-3A-A9IV,1.8566,1
54,TCGA-FB-AAPP,1.2802,1
60,TCGA-FB-AAQ1,0.741,1
168,TCGA-YY-A8LH,0.1962,1


Note that changes in expression levels are low, suggesting that copy number alterations may not affect SMAD4 levels by much

### 5. Mutual exclusivity and co-occurrence analysis
 Analyze your top 3 most mutated genes from above. Make a visualization that allows you to compare mutual exclusivity or co-occurrence for all pairs of any two of these mutations.

In [63]:
### function to calculate the number of patients that have a mutation in both of the genes of interest
def num_mut_in_both(total_mut_per_patient, gene1, gene2):
  ## create a dataframe with the data of interest
  genes_mut = total_mut_per_patient[['PATIENT_ID', gene1, gene2]]
  ## store boolean series stating whether each patient has a mutation in each gene
  mut_in_gene1 =  genes_mut[gene1] > 0
  mut_in_gene2 =  genes_mut[gene2] > 0
  ## add a column stating whether the patient has a mutation in both genes
  genes_mut['co-occurrence'] = mut_in_gene1 | mut_in_gene2
  ## store the total number patients that have a mutation in both genes
  num_mut = genes_mut['co-occurrence'].sum()

  return num_mut

In [107]:
# want to create a dataframe containing the number of samples that have mutations both of any 2 pairs of mutations --> heatmap
### create a new dataframe to store the number patients that have a mutation in any pair of genes
co_occurrence = pd.DataFrame(columns = top_genes)
### add a column with the gene and set it as the name the rows
co_occurrence['gene'] = top_genes
co_occurrence.set_index('gene', inplace=True)
### for each gene
for idx, gene1 in enumerate(top_genes):
  ## store the names of the other genes of interest
  other_genes = top_genes[:idx] + top_genes[idx+1:]
  ## iterate through the other genes
  for gene2 in other_genes:
    ## get the number of patients that have a mutation in both genes
    num_mut = num_mut_in_both(total_mut_per_patient, gene1, gene2)
    ## store the fraction of all samples that have mutation in both genes
    co_occurrence.loc[gene1, gene2] = num_mut/survival_outcome.shape[0]
### fill NAs with 1
co_occurrence.fillna(1, inplace=True)

In [108]:
# create a heatmap showing the number of patients that have a mutation in both genes
fig = px.imshow(co_occurrence, text_auto=True)

# add title, axis labels
fig.update_layout(title = 'Figure 6. Heatmap Showing Fraction of All Patients Exhibiting Both Mutations',
                  title_x = 0.5,
                  xaxis_title = '',
                  yaxis_title = '')

# move the x-axis labels to the top
fig.update_xaxes(side="top")

# show heatmap
fig.show()

Based on the above heatmap (Figure 6), at least 70% of all patients with a patient outcome had mutations in any pair of two genes.

### 6. Make one other plot
Make one other plot to learn about the effect of this pathway on this cancer, using any of the data you have about the cancer cohort. Meaning, a different from the ones already required, using coding.

#### a. Form a hypothesis of something you expect to see in this cancer data.

I would expect that if we look across all top 20 most mutated genes, all tumors would have similar gene expression patterns as they all have pancreatic adenocarcinoma. However, based on the initial analysis in question 5, it does not seem like expression levels would be similar across different genes.

#### b. Describe what kind of plot you choose and how it will help assess your hypothesis. What do you expect to see?

To compare similarity in expression patterns across tumor samples and genes, I could create a heatmap in which each row represent one of the top 20 most mutated genes and each column is a tumor sample. A column of the heatmap would show the expression levels across all 20 genes for one tumor sample. I would expect that the heatmaps be filled with mainly the same color (have similar gene expression levels) across all columns.

#### c. Provide code to make the plot. The code must work and must be explained so someone else can understand how it works

In [ ]:
# store the list of the top 20 genes
top_genes = num_samples_with_mut.head(20)['gene'].tolist()
# store the expression levels of these top 20 genes
top_gene_exp = mRNA_exp.loc[mRNA_exp['Hugo_Symbol'].isin(top_genes)]
# remove the gene id as a column
top_gene_exp.drop('Entrez_Gene_Id', axis=1, inplace=True)
# set index as the gene name
top_gene_exp.set_index('Hugo_Symbol', inplace=True)
# order the genes from greatest to least mutated
top_gene_exp = top_gene_exp.reindex(top_genes)
top_gene_exp

,TCGA-2J-AAB1-01,TCGA-2J-AAB4-01,TCGA-2J-AAB6-01,TCGA-2J-AAB8-01,TCGA-2J-AAB9-01,TCGA-2J-AABA-01,TCGA-2J-AABE-01,TCGA-2J-AABF-01,TCGA-2J-AABH-01,TCGA-2J-AABI-01,...,TCGA-XD-AAUG-01,TCGA-XD-AAUH-01,TCGA-XD-AAUI-01,TCGA-XD-AAUL-01,TCGA-XN-A8T3-01,TCGA-XN-A8T5-01,TCGA-YB-A89D-01,TCGA-YH-A8SY-01,TCGA-YY-A8LH-01,TCGA-Z5-AAPL-01
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
SMAD4,0.6559,-0.3810,-0.9210,-0.5721,0.3982,0.1223,-1.0689,0.0300,-0.4407,-1.6715,...,0.7784,0.8717,0.0676,0.0728,0.4426,0.1801,0.1570,-0.5816,0.1962,0.2616
CDH20,0.3970,-1.2635,-1.7781,-0.8073,-0.7111,-0.2293,0.9748,-0.4850,-0.5539,-0.9789,...,-0.1557,0.4712,0.4143,0.3674,-0.7231,-1.0934,1.2081,-0.0132,-0.8519,-1.7781
CDH7,-1.2063,-1.2063,-1.2063,-1.2063,-1.2063,-1.2063,-0.2218,-0.1081,-1.2063,-1.2063,...,-1.2063,-1.2063,-1.2063,-1.2063,-1.2063,-1.2063,-0.3563,-1.2063,-1.2063,-1.2063
CDH19,0.2986,-0.8133,-0.7142,1.2711,0.6211,2.2163,-1.1763,0.2235,-0.8286,-2.2097,...,1.2665,0.4317,0.8106,0.8247,-0.4600,2.3161,-0.0957,0.2518,0.6867,-2.4447
NFATC1,-0.3591,-0.5635,-1.0877,-0.2127,0.4923,-0.2435,0.0103,-0.4265,-1.2342,0.3180,...,2.4807,1.6227,0.2609,0.2668,-0.1698,2.2754,-0.4604,0.3489,0.0779,1.8340
CDH2,-0.9338,-0.3654,-0.1974,0.4873,0.6701,-0.9683,0.1910,0.3191,0.7448,-0.7870,...,0.7657,0.2799,-0.1906,1.0645,1.1233,0.1129,0.4415,-0.0367,-1.6637,-0.2182
CER1,-1.4371,-1.4371,-0.3941,-1.4371,0.2128,-1.4371,-1.4371,-0.7500,-1.4371,-1.4371,...,-0.2678,0.1790,-1.4371,-1.4371,-0.4777,-0.3783,0.1641,-1.4371,-1.4371,-1.4371
DVL2,0.8052,-0.4866,0.9132,-0.7051,0.1595,1.1582,-0.2698,-0.0332,-0.6621,0.1232,...,1.2088,0.4069,-0.6851,-0.1249,0.1759,0.1212,0.5913,-0.5814,-1.4447,0.1561
ARRB2,0.1354,0.1021,-0.0126,-0.5458,0.5620,-0.7619,0.1072,1.0679,-0.4479,-0.2727,...,0.1986,0.7234,-0.0934,0.0147,0.0373,0.2134,0.3988,-0.1740,-1.9899,1.2976


In [ ]:
# perform hierarchical clustering of genes and tumor samples
dash_bio.Clustergram(
    data = top_gene_exp,
    column_labels=list(top_gene_exp.columns.values),
    row_labels = list(top_gene_exp.index),
    height=700,
    width=1000)

#### d. Interpret the plot: how does the plot allow you to assess your hypothesis? What do you conclude by looking at this.

### 7. Another Research Article
Find one other research article related to this type of cancer that refers to either: the activity of this pathway; or the genes/mutations in the pathway; or a drug and how it can impact that pathway in that cancer.

Research article: https://www.nature.com/articles/nature14169#Sec13
#### a. Summarize article
Describe the:
* knowledge gap
* hypothesis
* experimental design of the paper (what are the observations and how many observations; and what are the variables and approximately how many)


I will explain the summarize the goal of the paper, their hypothesis, and general idea of how they test this hypothesis

#### b. Summarize how the findings in the article relates to your findings above about the pathway

I will explain what they found about the SMAD4 and how that relates to results that I found in my analysis.

#### c. Include the two most relevant figures from the paper!
For each figure:
* explain what are the axes (horizontal and vertical)
* what the points, bars, colors mean (depending on the plot. Please see me if you are not sure which parts to explain)
* Copy over one sentence (quote the sentence) showing a conclusions they made based on this figure

#### [Figure 1](https://drive.google.com/file/d/1fnd5Z-M96Hh35bEJ0TpdT-qirgkMraTd/view?usp=sharing)

I will explain all components of the upper panel and the central matrix before explaining how the figure supports the conclusion made in the paper.

**Conclusion from the paper:** "Chromothripsis is linked to TP53 mutations in medullobastoma and acute myeloid leukaemia and here, 10/13 chromothriptic tumours had a TP53 mutation, 5 of which were bi-allelic." [[6]](https://www.nature.com/articles/nature14169#Sec13)

#### [Figure 2b](https://drive.google.com/file/d/12wUsg7ffRCCLgqfEUOBA9VwRcPobR1Vy/view?usp=sharing)

I will explain all components of the upper panel and the central matrix before explaining how the figure supports the conclusion made in the paper.

**Conclusion from paper:** "Unstable tumours are associated with a high BRCA mutation signature and deleterious mutations in BRCA pathway genes." [[6]](https://www.nature.com/articles/nature14169#Sec13)

### References:
[1] https://gis.cdc.gov/Cancer/USCS/#/AtAGlance/

[2] https://www.cancer.org/cancer/types/pancreatic-cancer/about/what-is-pancreatic-cancer.html

[3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5357762/

[4] https://www.cbioportal.org/study/summary?id=paad_tcga_pan_can_atlas_2018

[5] https://maayanlab.cloud/Harmonizome/gene_set/Wnt+signaling+pathway/PANTHER+Pathways)

[6] https://www.nature.com/articles/nature14169#Sec13